In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
file_path = '/content/drive/MyDrive/Colab/NU/IE6200/stations.csv'
stations_df = pd.read_csv(file_path, skiprows=1, header=0,
                          usecols=['Number', 'NAME', 'Lat', 'Long',
                                   'Municipality', 'Total Docks'],
                          nrows=11)
stations_df.head()

,Number,NAME,Lat,Long,Municipality,Total Docks
0,L32001,Railroad Lot and Minuteman Bikeway,42.416065,-71.153366,Arlington,11
1,L32002,Linwood St at Minuteman Bikeway,42.409354,-71.149065,Arlington,11
2,L32005,Thorndike Field at Minuteman Bikeway,42.400168,-71.144570,Arlington,11
3,L32003,Mass Ave at Grafton St,42.407261,-71.143821,Arlington,11
4,L32004,Broadway at Grafton St,42.409942,-71.140093,Arlington,11


In [3]:
# Create GeoDataFrame with station locations
stations_df['geometry'] = stations_df.apply(lambda row: Point(row['Long'], row['Lat']), axis=1)
stations_gdf = gpd.GeoDataFrame(stations_df, geometry='geometry', crs="EPSG:4326")

# Define area of interest around stations
convex_hull = stations_gdf.unary_union.convex_hull.buffer(0.02)

# Get road network from OpenStreetMap
G = ox.graph_from_polygon(convex_hull, network_type='drive')

# Project graph to UTM for distance calculations
G_proj = ox.project_graph(G)

# Project stations to the same CRS as the graph
stations_gdf = stations_gdf.to_crs(G_proj.graph['crs'])

stations_gdf.head()

,Number,NAME,Lat,Long,Municipality,Total Docks,geometry
0,L32001,Railroad Lot and Minuteman Bikeway,42.416065,-71.153366,Arlington,11,POINT (322825.928 4698219.344)
1,L32002,Linwood St at Minuteman Bikeway,42.409354,-71.149065,Arlington,11,POINT (323160.974 4697465.203)
2,L32005,Thorndike Field at Minuteman Bikeway,42.400168,-71.144570,Arlington,11,POINT (323505.154 4696435.812)
3,L32003,Mass Ave at Grafton St,42.407261,-71.143821,Arlington,11,POINT (323586.623 4697221.832)
4,L32004,Broadway at Grafton St,42.409942,-71.140093,Arlington,11,POINT (323900.925 4697511.870)


In [4]:
# Function to calculate a distance matrix
def calculate_distance_matrix(graph, stations_gdf):
    # Get station numbers and coordinates
    station_numbers = stations_gdf['Number'].tolist()
    station_coords = list(zip(stations_gdf.geometry.y, stations_gdf.geometry.x))

    # Initialize an empty DataFrame for the matrix
    distance_matrix = pd.DataFrame(index=station_numbers, columns=station_numbers, dtype=float)

    # Iterate through each pair of stations
    for i, (number1, coord1) in enumerate(zip(station_numbers, station_coords)):
        for j, (number2, coord2) in enumerate(zip(station_numbers, station_coords)):
            if number1 != number2:  # Avoid self-loops
                # Find nearest nodes on the graph
                orig_node = ox.distance.nearest_nodes(graph, coord1[1], coord1[0])
                dest_node = ox.distance.nearest_nodes(graph, coord2[1], coord2[0])

                # Check if a path exists between the nodes
                if nx.has_path(graph, orig_node, dest_node):
                    # Calculate shortest path length
                    path_length = nx.shortest_path_length(graph, orig_node, dest_node, weight='length')
                    distance_matrix.at[number1, number2] = path_length
                else:
                    # No path available
                    distance_matrix.at[number1, number2] = None

    return distance_matrix

# Calculate the distance matrix
distance_matrix = calculate_distance_matrix(G_proj, stations_gdf)

# Display the distance matrix
distance_matrix

,L32001,L32002,L32005,L32003,L32004,L32006,A32000,A32001,A32002,A32003,A32004
L32001,NaN,1269.466,2279.787,1469.433,1557.933,494.204,12288.900,9296.165,9794.830,9310.272,11040.710
L32002,1119.835,NaN,1181.256,520.287,913.384,789.754,11339.754,8347.019,8845.684,8361.126,10091.564
L32005,2130.156,1181.256,NaN,855.130,1302.636,1800.075,10758.852,7766.117,8264.782,7780.224,9510.662
L32003,1319.802,520.287,857.504,NaN,458.030,989.721,10819.467,7826.732,8325.397,7840.839,9571.277
L32004,1336.731,918.502,1412.527,555.023,NaN,1006.650,10913.794,8139.436,8638.101,8153.543,9883.981
L32006,330.081,800.370,1810.691,1000.337,1088.837,NaN,11819.804,8827.069,9325.734,8841.176,10571.614
A32000,12225.769,11426.254,10845.352,10905.967,10896.792,11895.688,NaN,8506.108,6927.456,6326.381,6246.251
A32001,9119.287,8319.772,7738.870,7799.485,8172.185,8789.206,8225.287,NaN,1596.269,2334.370,3385.465
A32002,9609.640,8810.125,8229.223,8289.838,8662.538,9279.559,6673.022,1578.652,NaN,782.105,2094.204
A32003,9065.255,8265.740,7684.838,7745.453,8118.153,8735.174,5890.917,2348.350,769.698,NaN,1812.241


In [ ]:
# Save the matrix to a CSV file
distance_matrix.to_csv("/content/drive/MyDrive/Colab/NU/IE6200/station_distance_matrix.csv")